In [1]:
import os
import logging
import gzip
from gensim.models import Word2Vec
import tensorflow as tf
import nltk
import pickle
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
from Word2VecUtility import Word2VecUtility
from keras.callbacks import TensorBoard
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv1D, MaxPooling1D
from gensim.models import word2vec
from data_helpers import clean_str, load_data_and_labels, batch_iter


C:\Users\Rizary\AppData\Roaming\Python\Python36\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
d:\engineer\ide\anaconda3\envs\tensorflow\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [2]:

def get_volcabulary_and_list_words(data):
    reviews_words = []
    volcabulary = []
    for review in data["content"]:
        review_words = Word2VecUtility.review_to_wordlist(
            review, remove_stopwords=True)
        reviews_words.append(review_words)
        for word in review_words:
            volcabulary.append(word)
    volcabulary = set(volcabulary)
    return volcabulary, reviews_words



In [3]:
def get_reviews_word_index(reviews_words, volcabulary, max_words, max_length):
    word2index = {word: i for i, word in enumerate(volcabulary)}
    # use w in volcabulary to limit index within max_words
    reviews_words_index = [[start] + [(word2index[w] + index_from) for w in review] for review in reviews_words]
    # in word2vec embedding, use (i < max_words + index_from) because we need the exact index for each word, in order to map it to its vector. And then its max_words is 5003 instead of 5000.
    reviews_words_index = [[i if (i < max_words) else oov for i in index] for index in reviews_words_index]
    # padding with 0, each review has max_length now.
    reviews_words_index = sequence.pad_sequences(reviews_words_index, maxlen=max_length, padding='post', truncating='post')
    return reviews_words_index

In [4]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [5]:
df=pd.read_json('./file.json')

In [6]:
nltk.download('stopwords')
getVocab = get_volcabulary_and_list_words(df)
print(getVocab)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rizary\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


d:\engineer\ide\anaconda3\envs\tensorflow\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file d:\engineer\ide\anaconda3\envs\tensorflow\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


({'insist', 'problem', 'favor', 'drill', 'bank', 'strip', 'popular', 'return', 'mean', 'lemond', 'german', 'downtown', 'extraordinarili', 'cali', 'januari', 'histori', 'worth', 'malnutrit', 'kondow', 'outrag', 'harder', 'alway', 'reuven', 'cynic', 'game', 'stroller', 'coolest', 'alic', 'oust', 'birthday', 'ultim', 'technolog', 'tour', 'seeker', 'beattheheatindia', 'close', 'multi', 'strand', 'cnn', 'talent', 'volgograd', 'routin', 'ahead', 'london', 'hydrat', 'use', 'app', 'senat', 'bharatiya', 'lucr', 'tie', 'watch', 'social', 'ammunit', 'incom', 'famou', 'award', 'relinquish', 'corp', '2019', 'tech', 'grade', 'oper', 'assess', 'mauric', 'boss', 'test', 'correctli', 'tape', 'program', 'suppli', 'vulner', 'ubiquit', 'hear', 'zimbabw', 'panel', 'sign', 'center', 'sixteen', 'futur', 'told', 'begin', 'video', 'intimaci', 'reduc', 'cliqu', 'crime', 'excit', 'obstacl', 'ritz', 'show', 'ecuadorian', 'suffer', 'gone', 'laurent', 'defin', 'young', 'paper', 'cann', 'environ', 'yard', 'injuri', 

In [7]:
#title = df['title'].values.tolist()
#content = df['content'].values.tolist()

In [8]:
#title[4]

In [9]:
#data processing para
max_words = 5000
max_length = 50

#model training parameters
batch_size= 32
embedding_dims = 100
nb_filter = 250
filter_length = 3
hidden_dims = 250
nb_epoch = 2

#index trick parameters
index_from = 3
start = 1
#padding = 0
oov = 2


In [10]:
print ('Get volcabulary...')
volcabulary, news_words = get_volcabulary_and_list_words(df)

Get volcabulary...


d:\engineer\ide\anaconda3\envs\tensorflow\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file d:\engineer\ide\anaconda3\envs\tensorflow\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


In [11]:
print('get reviews_words_index...')
reviews_words_index = get_reviews_word_index(news_words, volcabulary, max_words, max_length)

get reviews_words_index...


In [12]:
print (reviews_words_index[:20, :12])
print (reviews_words_index.shape)

[[   1  942 2063  440  760  188 1068 1113 2442 2222  375  828]
 [   1  368  662  436  767  580 1356 1686  835  441 2249 1018]
 [   1  284 1380  194  997 1046 1052  177  551  188 1068 1046]
 [   1  243 1722  236 1298 1837 2057  966  840 2197 1820  984]
 [   1 2292 1388   35  683 1538  611 2635  216  958  656 1749]
 [   1  234 1706 2206  574 1193  874 1706 2359 1857   41  736]
 [   1 1081  727 2248 1369 2594  915  450  188 1334  142  542]
 [   1  571 1591  957 1941 1512  978 1272 1549 1099 2550 2455]
 [   1 2272 2085 2276 1122 2339 2553 2042 1904  299 1383 2461]
 [   1 1549 1909 2322 1161  331 1505  683 1689  665 2016  917]
 [   1 2633 2005 1129  123 2377  583 1658 2426  116 1681  116]
 [   1  293 2114 2323 1788 2087   48 1965 1991  731   99  611]
 [   1  976  810 2206  157  211 2152  385   58 1056  588 2302]
 [   1  542 2255 1354 2144  354 2555   45 1175 2200  984  683]
 [   1 1861 1859  841 1272 2467  644 1208  827 1620 1732 1926]
 [   1 1394 1827  426 2238  433  485 1727 2057 2418 120

In [13]:
index = np.arange(reviews_words_index.shape[0])
train_index, valid_index = train_test_split(
    index, train_size=0.8, random_state=520)

In [14]:
train_data = reviews_words_index[train_index]
valid_data = reviews_words_index[valid_index]
print (train_data.shape)
print (valid_data.shape)

(25, 50)
(7, 50)


In [15]:


print ("start training model...")

model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_words + index_from, embedding_dims, \
                    input_length=max_length))
model.add(Dropout(0.25))

# we add a Convolution1D, which will learn nb_filter
# word group filters of size filter_length:

# filter_length is like filter size, subsample_length is like step in 2D CNN.
model.add(Conv1D(filters=nb_filter,
                        kernel_size=filter_length,
                        padding='valid',
                        activation='relu',
                        strides=1))
# we use standard max pooling (halving the output of the previous layer):
model.add(MaxPooling1D(pool_size=2))

# We flatten the output of the conv layer,
# so that we can add a vanilla dense layer:
model.add(Flatten())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.25))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              class_mode='binary')

start training model...


In [16]:
tbCallback = TensorBoard(log_dir='D:\Engineer\Work\DeepLearning\logs_1', histogram_freq=0,  
          write_graph=True, write_images=True)

In [17]:

#model.fit(train_data, batch_size=batch_size,
#          epochs=nb_epoch, callbacks=tbCallback)

In [18]:
# this is second attempt to use word2vec instead
print ('\nThe first news is:\n')
print (df["content"][0], '\n')
print (df.shape)
print (df.columns)


The first news is:

The team's loss against Croatia meant they also lost out on another $50,000 promised to them by the federation sponsor Aiteo, while Nigeria's Senate President Bukola Saraki also promised the team another $50,000 for all games won, according to NFF president Amaju Pinnick.   Reports from local media also said Saraki   ahead of their departure to Russia. Bonus payments have been an issue of controversy within the Nigeria squad and some other African countries in the past.  Nigerian players have battled with the NFF leadership over very long delays in paying out bonuses especially in the qualification games leading to major tournaments.  The fight culminated in a series of strikes by players at the start of the 2013 FIFA Confederations Cup and during the 2014 World Cup in Brazil. Ghana's Black Stars made global headlines for the wrong reasons at the 2014 World Cup in Brazil after their fight with their federation over monies owed.  The disagreement forced FIFA to inte

In [19]:
print (df['title'][:3])
print ('\n')
print (df.loc[:2]['content'])

0    Nigeria's Super Eagles miss out on nearly $300...
1    Nigerian singer D'banj loses son in swimming p...
2    Why we have fallen hard for the Senegal World ...
Name: title, dtype: object


0    The team's loss against Croatia meant they als...
1    The singer regularly shared pictures of his so...
2    Egypt are almost certainly out of the tourname...
Name: content, dtype: object


In [20]:
print (df.shape)
print (df.columns)
print (df.index)
# only after to_csv without index, and read_csv back to data, can you use ix[5]
# print data.ix[:5]['text']
# if you want to index dataframe directly after sampled it. use iloc
print (df.iloc[:5])

(32, 2)
Index(['content', 'title'], dtype='object')
RangeIndex(start=0, stop=32, step=1)
                                             content  \
0  The team's loss against Croatia meant they als...   
1  The singer regularly shared pictures of his so...   
2  Egypt are almost certainly out of the tourname...   
3  Lalong called the curfew "an immediate measure...   
4  , the Prime Minister has toured the country, l...   

                                               title  
0  Nigeria's Super Eagles miss out on nearly $300...  
1  Nigerian singer D'banj loses son in swimming p...  
2  Why we have fallen hard for the Senegal World ...  
3  Nigeria: Scores killed, homes burned in Platea...  
4  Ethiopia's new Prime Minister has had a stella...  


In [21]:
import nltk.data
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [22]:
# print data.ix[0:10]
print (df.iloc[1]['content'])
# print data['text'][2]

The singer regularly shared pictures of his son on his Instagram page and posted an image of him changing Daniel's soiled diapers on Father's Day.  Meanwhile the Lagos state police command released a statement saying a delegation of officers had visited the family at their home in the Ikoyi suburb of Lagos on Monday but no one was home.  The statement asked D'banj and his wife to report the circumstances of the boy's death to the police station after their mourning period.  "It is expected that when the mourning period is over, the family should formally report the circumstances that led to the unfortunate death of little Daniel to the police. May his soul rest in peace. Amen," police spokesman Chike Oti wrote in the statement.  Celebrities and fans around the world have been posting their condolence messages to the family on social media.  Another Afropop star, Davido, who won the BET Awards' Best International Act on Sunday, offered his condolences to the star while receiving his awa

In [23]:
review_sents = []
print ("Cleaning and parsing the articles...\n")
for i in range(0, len(df["content"])):
    # sent_reviews += Word2VecUtility.review_to_sentences(data["text"][i], tokenizer)
    review_sents += Word2VecUtility.review_to_sentences(df.iloc[i]["content"], tokenizer)

Cleaning and parsing the articles...



d:\engineer\ide\anaconda3\envs\tensorflow\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file d:\engineer\ide\anaconda3\envs\tensorflow\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))
d:\engineer\ide\anaconda3\envs\tensorflow\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


In [24]:
print(review_sents)

[LabeledSentence(words=['the', 'team', 's', 'loss', 'against', 'croatia', 'meant', 'they', 'also', 'lost', 'out', 'on', 'anoth', '50', '000', 'promis', 'to', 'them', 'by', 'the', 'feder', 'sponsor', 'aiteo', 'while', 'nigeria', 's', 'senat', 'presid', 'bukola', 'saraki', 'also', 'promis', 'the', 'team', 'anoth', '50', '000', 'for', 'all', 'game', 'won', 'accord', 'to', 'nff', 'presid', 'amaju', 'pinnick'], tags=['sentence0']), LabeledSentence(words=['report', 'from', 'local', 'media', 'also', 'said', 'saraki', 'ahead', 'of', 'their', 'departur', 'to', 'russia'], tags=['sentence1']), LabeledSentence(words=['bonu', 'payment', 'have', 'been', 'an', 'issu', 'of', 'controversi', 'within', 'the', 'nigeria', 'squad', 'and', 'some', 'other', 'african', 'countri', 'in', 'the', 'past'], tags=['sentence2']), LabeledSentence(words=['nigerian', 'player', 'have', 'battl', 'with', 'the', 'nff', 'leadership', 'over', 'veri', 'long', 'delay', 'in', 'pay', 'out', 'bonus', 'especi', 'in', 'the', 'qualif'

In [25]:
print (len(review_sents))
print (review_sents[:5])

643
[LabeledSentence(words=['the', 'team', 's', 'loss', 'against', 'croatia', 'meant', 'they', 'also', 'lost', 'out', 'on', 'anoth', '50', '000', 'promis', 'to', 'them', 'by', 'the', 'feder', 'sponsor', 'aiteo', 'while', 'nigeria', 's', 'senat', 'presid', 'bukola', 'saraki', 'also', 'promis', 'the', 'team', 'anoth', '50', '000', 'for', 'all', 'game', 'won', 'accord', 'to', 'nff', 'presid', 'amaju', 'pinnick'], tags=['sentence0']), LabeledSentence(words=['report', 'from', 'local', 'media', 'also', 'said', 'saraki', 'ahead', 'of', 'their', 'departur', 'to', 'russia'], tags=['sentence1']), LabeledSentence(words=['bonu', 'payment', 'have', 'been', 'an', 'issu', 'of', 'controversi', 'within', 'the', 'nigeria', 'squad', 'and', 'some', 'other', 'african', 'countri', 'in', 'the', 'past'], tags=['sentence2']), LabeledSentence(words=['nigerian', 'player', 'have', 'battl', 'with', 'the', 'nff', 'leadership', 'over', 'veri', 'long', 'delay', 'in', 'pay', 'out', 'bonus', 'especi', 'in', 'the', 'qua

In [26]:
# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
#print ("Training model...")
#model = word2vec.Word2Vec(review_sents, workers=num_workers, \
#            size=num_features, min_count = min_word_count, \
#            window = context, sample = downsampling)

In [27]:
import re
import itertools
from collections import Counter


def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()

In [28]:
def load_data_and_labels(positive_data_file, negative_data_file):
    """
    Loads MR polarity data from files, splits the data into words and generates labels.
    Returns split sentences and labels.
    """
    # Load data from files
    positive_examples = list(open(positive_data_file, "r", encoding='utf-8').readlines())
    positive_examples = [s.strip() for s in positive_examples]
    negative_examples = list(open(negative_data_file, "r", encoding='utf-8').readlines())
    negative_examples = [s.strip() for s in negative_examples]
    # Split by words
    x_text = positive_examples + negative_examples
    x_text = [clean_str(sent) for sent in x_text]
    # Generate labels
    positive_labels = [[0, 1] for _ in positive_examples]
    negative_labels = [[1, 0] for _ in negative_examples]
    y = np.concatenate([positive_labels, negative_labels], 0)
    return [x_text, y]

In [29]:
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
    yield shuffled_data[start_index:end_index]

In [30]:
tf.flags.DEFINE_string("positive_data_file", "./data/rt-polaritydata/rt-polarity.pos", "Data source for the positive data.")
tf.flags.DEFINE_string("negative_data_file", "./data/rt-polaritydata/rt-polarity.neg", "Data source for the negative data.")

In [31]:
last = df.reindex(np.random.permutation(df.index))
labels = sorted(list(set(df["title"].tolist())))
one_hot = np.zeros((len(labels), len(labels)), int)
np.fill_diagonal(one_hot,1)
#print(labels)

In [32]:
label_dict = dict(zip(labels,one_hot))
#print(label_dict)

In [33]:
x_raw = df["content"].apply(lambda x: clean_str(x)).tolist()
y_raw = df["title"].apply(lambda y: label_dict[y]).tolist()
#print(y_raw)
#print(x_raw)

In [34]:
import os
import sys
import json
import time
import logging
import data_helpers
import numpy as np
import tensorflow as tf
from text_cnn import TextCNN
from tensorflow.contrib import learn
from sklearn.model_selection import train_test_split

In [35]:
params = json.loads(open('parameters.json').read())

In [36]:
"""Step 1: pad each sentence to the same length and map each word to an id"""
max_document_length = max([len(x.split(' ')) for x in x_raw])
logging.info('The maximum length of all sentences: {}'.format(max_document_length))
vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
x = np.array(list(vocab_processor.fit_transform(x_raw)))
y = np.array(y_raw)

2018-07-03 09:23:03,465 : INFO : The maximum length of all sentences: 1015


Instructions for updating:
Please use tensorflow/transform or tf.data.


2018-07-03 09:23:03,519 : WARNING : From <ipython-input-36-1600b76d511a>:4: VocabularyProcessor.__init__ (from tensorflow.contrib.learn.python.learn.preprocessing.text) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tensorflow/transform or tf.data.


Instructions for updating:
Please use tensorflow/transform or tf.data.


2018-07-03 09:23:03,539 : WARNING : From d:\engineer\ide\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\contrib\learn\python\learn\preprocessing\text.py:154: CategoricalVocabulary.__init__ (from tensorflow.contrib.learn.python.learn.preprocessing.categorical_vocabulary) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tensorflow/transform or tf.data.


Instructions for updating:
Please use tensorflow/transform or tf.data.


2018-07-03 09:23:03,561 : WARNING : From d:\engineer\ide\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\contrib\learn\python\learn\preprocessing\text.py:170: tokenizer (from tensorflow.contrib.learn.python.learn.preprocessing.text) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tensorflow/transform or tf.data.


In [37]:
#print(x)
#print(y)

In [38]:
"""Step 2: split the original dataset into train and test sets"""
x_, x_test, y_, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

In [39]:
"""Step 3: shuffle the train set and split the train set into train and dev sets"""
shuffle_indices = np.random.permutation(np.arange(len(y_)))
x_shuffled = x_[shuffle_indices]
y_shuffled = y_[shuffle_indices]
x_train, x_dev, y_train, y_dev = train_test_split(x_shuffled, y_shuffled, test_size=0.1)

In [40]:
"""Step 4: save the labels into labels.json since predict.py needs it"""
with open('./labels.json', 'w') as outfile:
    json.dump(labels, outfile, indent=4)

logging.info('x_train: {}, x_dev: {}, x_test: {}'.format(len(x_train), len(x_dev), len(x_test)))
logging.info('y_train: {}, y_dev: {}, y_test: {}'.format(len(y_train), len(y_dev), len(y_test)))

2018-07-03 09:23:17,883 : INFO : x_train: 25, x_dev: 3, x_test: 4
2018-07-03 09:23:17,884 : INFO : y_train: 25, y_dev: 3, y_test: 4


In [47]:
"""Step 5: build a graph and cnn object"""
graph = tf.Graph()
with graph.as_default():
        session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
        sess = tf.Session(config=session_conf)
        with sess.as_default():
                cnn = TextCNN(
                        sequence_length=x_train.shape[1],
                        num_classes=y_train.shape[1],
                        vocab_size=len(vocab_processor.vocabulary_),
                        embedding_size=params['embedding_dim'],
                        filter_sizes=list(map(int, params['filter_sizes'].split(","))),
                        num_filters=params['num_filters'],
                        l2_reg_lambda=params['l2_reg_lambda'])

                global_step = tf.Variable(0, name="global_step", trainable=False)
                optimizer = tf.train.AdamOptimizer(1e-3)
                grads_and_vars = optimizer.compute_gradients(cnn.loss)
                train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

                timestamp = str(int(time.time()))
                out_dir = os.path.abspath(os.path.join(os.path.curdir + "/copy_train", "trained_model_" + timestamp))
                print("Writing to {}\n".format(out_dir))
            
                checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
                checkpoint_prefix = os.path.join(checkpoint_dir, "model")
                if not os.path.exists(checkpoint_dir):
                        os.makedirs(checkpoint_dir)
                saver = tf.train.Saver()

                # One training step: train the model with one batch
                def train_step(x_batch, y_batch):
                        feed_dict = {
                                cnn.input_x: x_batch,
                                cnn.input_y: y_batch,
                                cnn.dropout_keep_prob: params['dropout_keep_prob']}
                        _, step, loss, acc = sess.run([train_op, global_step, cnn.loss, cnn.accuracy], feed_dict)

                # One evaluation step: evaluate the model with one batch
                def dev_step(x_batch, y_batch):
                        feed_dict = {cnn.input_x: x_batch, cnn.input_y: y_batch, cnn.dropout_keep_prob: 1.0}
                        step, loss, acc, num_correct = sess.run([global_step, cnn.loss, cnn.accuracy, cnn.num_correct], feed_dict)
                        return num_correct

                # Save the word_to_id map since predict.py needs it
                vocab_processor.save(os.path.join(out_dir, "vocab.pickle"))
                sess.run(tf.global_variables_initializer())

                # Training starts here
                train_batches = data_helpers.batch_iter(list(zip(x_train, y_train)), params['batch_size'], params['num_epochs'])
                best_accuracy, best_at_step = 0, 0

                """Step 6: train the cnn model with x_train and y_train (batch by batch)"""
                for train_batch in train_batches:
                        x_train_batch, y_train_batch = zip(*train_batch)
                        train_step(x_train_batch, y_train_batch)
                        current_step = tf.train.global_step(sess, global_step)

                        """Step 6.1: evaluate the model with x_dev and y_dev (batch by batch)"""
                        if current_step % params['evaluate_every'] == 0:
                                dev_batches = data_helper.batch_iter(list(zip(x_dev, y_dev)), params['batch_size'], 1)
                                total_dev_correct = 0
                                for dev_batch in dev_batches:
                                    x_dev_batch, y_dev_batch = zip(*dev_batch)
                                    num_dev_correct = dev_step(x_dev_batch, y_dev_batch)
                                    total_dev_correct += num_dev_correct

                                dev_accuracy = float(total_dev_correct) / len(y_dev)
                                logging.critical('Accuracy on dev set: {}'.format(dev_accuracy))

                                """Step 6.2: save the model if it is the best based on accuracy on dev set"""
                                if dev_accuracy >= best_accuracy:
                                        best_accuracy, best_at_step = dev_accuracy, current_step
                                        path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                                        logging.critical('Saved model at {} at step {}'.format(path, best_at_step))
                                        logging.critical('Best accuracy is {} at step {}'.format(best_accuracy, best_at_step))

                """Step 7: predict x_test (batch by batch)"""
                test_batches = data_helpers.batch_iter(list(zip(x_test, y_test)), params['batch_size'], 1)
                total_test_correct = 0
                for test_batch in test_batches:
                        x_test_batch, y_test_batch = zip(*test_batch)
                        num_test_correct = dev_step(x_test_batch, y_test_batch)
                        total_test_correct += num_test_correct

                test_accuracy = float(total_test_correct) / len(y_test)
                #logging.critical('Accuracy on test set is {} based on the best model {}'.format(test_accuracy, path))
                logging.critical('The training is complete')

Writing to D:\Engineer\Work\DeepLearning\copy_train\trained_model_1530585447



2018-07-03 09:37:28,193 : CRITICAL : The training is complete
